### Import Required Libraries and Set Up Environment Variables

In [6]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [78]:
# Set environment variables from the .env in the local environment
load_dotenv("API_Key.env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [8]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# Arnab's comments: Build the NYT Article search URL using the above varaiables and API Keys in the f-string method
search_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&sort={sort}"


In [9]:
# Create an empty list to store the reviews
list_reviews = []

# loop through pages 0-19
for pages in range(0, 20):
    # create query with a page number
    # API results show 10 articles at a time
    query_search_by_page_url = f"{search_url}&page={pages}"
    
    # Make a "GET" request and retrieve the JSON
    json_search_results = requests.get(query_search_by_page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in json_search_results["response"]["docs"]:
            list_reviews.append(review)

        # Print the page that was just retrieved
        print(f"Checked page {pages}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"No results for page: {pages}")
        break;


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [10]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(list_reviews[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [11]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(list_reviews)
df_reviews.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [12]:
# Arnab's comments: Some insights about title data in "headline.main" column

# Find the count of records where "headline.main" column starts with "Review:"
title_count_start_with_Review = df_reviews[df_reviews["headline.main"].str.startswith("Review: ")].count()["headline.main"]
print(f"Number of reviews that start with 'Review: ' {title_count_start_with_Review}")

# Find the count of records where "headline.main" column starts with \u2018 and
# contains \u2019 followed by  "Review:"
title_count_start_with_u2018 = df_reviews[ 
                        df_reviews["headline.main"].str.startswith("\u2018") 
                        & 
                        df_reviews["headline.main"].str.contains("\u2019 Review:")
                        ].count()["headline.main"]
print(f"Number of reviews that start with \"\u2018\" and ends with \"\u2019 Review:\" {title_count_start_with_u2018}")

# Find the count of records where "headline.main" column is not null
title_count_not_null = df_reviews[df_reviews["headline.main"].notnull()].count()["headline.main"]
print(f"Number of reviews where Title is not null {title_count_not_null}")


Number of reviews that start with 'Review: ' 59
Number of reviews that start with "‘" and ends with "’ Review:" 141
Number of reviews where Title is not null 200


In [88]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

def get_title(headline):
    pattern_start_with_u2018 = '\u2018(.+?)\u2019 Review:'
    pattern_start_with_Review = 'Review:.*? \u2018(.+?)\u2019 .*?'

    match_start_with_u2018 = re.search(pattern_start_with_u2018, headline)
    match_start_with_Review = re.search(pattern_start_with_Review, headline)

    if match_start_with_u2018:
        return match_start_with_u2018.group(1)
    
    elif match_start_with_Review:
        return match_start_with_Review.group(1)

    else:
        return None

# Arnab's comments: I am using the re module for searching the title, its a very powerful tool to search for patterns in strings
# Using this, I am able to extract the exact movie title from the "headline.main" when the string starts with "Review: " or with unicode characters \u2018.
# When "headline.main" starts with "Review: ", I am using space before \u2018 and space after \u2019, this is to differentiate whether \u2019  is used to end the title 'or' used as a punctuation mark (apostrophe like What’s)

# e.g. in the 2nd record headline.main = "Review: ‘What’s Love Got to Do With It?’ Probably a Lot". Here I am correctly able to extract the title "What’s Love Got to Do With It?"
# whereas the sample you provided, the extracted title is "What’s Love Got to Do With It? Probably a Lot" which is incorrect. and 
# The tiltle will not match with the movie database to get the movie details.

In [24]:
df_reviews["title"] = df_reviews["headline.main"].apply(get_title)

# Print the first 5 records from the "title" column in the df_reviews DataFrame
print(df_reviews["title"].head())

0            The Attachment Diaries
1    What’s Love Got to Do With It?
2              You Can Live Forever
3         A Tourist’s Guide to Love
4           Other People’s Children
Name: title, dtype: object


In [23]:

# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

In [25]:
# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)
df_reviews.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [27]:
# Create a list from the "title" column using to_list()
list_titles = df_reviews["title"].to_list()

# Print the first 5 titles in the list
print(list_titles[:5])

# These titles will be used in the query for The Movie Database


['The Attachment Diaries', 'What’s Love Got to Do With It?', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children']


### Access The Movie Database API

In [79]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

# Use tmdb token to access the API
tmdb_token = f"Bearer {os.getenv('TMDB_API_TOKEN')}"
tmdb_headers = {
    "accept": "application/json",
    "Authorization": tmdb_token
}

In [80]:
# Create an empty list to store the results
list_tmdb_movies = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in list_titles:

    # Check if no title is found, then continue to the next title
    if title == None:
        continue

    # Check if we need to sleep before making a request
    if request_counter % (50+1) == 0:
        time.sleep(12)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    movie_search_response = requests.get(url + title + tmdb_key_string).json()


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie is not found
    try:
        if( len(movie_search_response['results']) > 0 ):

            # Get the movie id
            movie_id = movie_search_response['results'][0]['id']

            # Make a request for a the full movie details
            movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}"


            # Execute "GET" request with url
            movie_details_response = requests.get(movie_details_url, headers=tmdb_headers).json()

        
            # Extract the genre names into a list
            list_genres = [ genre['name'] for genre in movie_details_response['genres'] ]


            # Extract the spoken_languages' English name into a list
            list_spoken_languages = [ language['english_name'] for language in movie_details_response['spoken_languages'] ]


            # Extract the production_countries' name into a list
            list_production_countries = [ country['name'] for country in movie_details_response['production_countries'] ]


            # Add the relevant data to a dictionary
            dict_movie_detail = {
                "title" : title, 
                "original_title": movie_details_response["original_title"], 
                "overview" :  movie_details_response["overview"], 
                "homepage" :  movie_details_response["homepage"], 
                "genres" :  list_genres,
                "popularity" :  movie_details_response["popularity"], 
                "original_language" :  movie_details_response["original_language"], 
                "spoken_languages" :  list_spoken_languages,
                "production_countries" : list_production_countries,
                "runtime" :  movie_details_response["runtime"], 
                "budget" :  movie_details_response["budget"], 
                "revenue" :  movie_details_response["revenue"], 
                "release_date" :  movie_details_response["release_date"], 
                "vote_average" :  movie_details_response["vote_average"], 
                "vote_count" :  movie_details_response["vote_count"]
                }


            # append it to the tmdb_movies_list list
            list_tmdb_movies.append(dict_movie_detail)

            # Print out the title that was found
            print(f"Found {title}")

        else:
            print(f"{title} not found.")

    except Exception as ex:
        print(f"{title} not found or errored with an exception {ex}")

Found The Attachment Diaries
Found What’s Love Got to Do With It?
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
Found Cyrano
Found Th

In [81]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(list_tmdb_movies[:5], indent=4))



[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "homepage": "",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "popularity": 0.964,
        "original_language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ],
        "runtime": 102,
        "budget": 0,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 3
 

In [82]:
# Convert the results to a DataFrame
df_movies = pd.DataFrame(list_tmdb_movies)
df_movies.head()

,title,original_title,overview,homepage,genres,popularity,original_language,spoken_languages,production_countries,runtime,budget,revenue,release_date,vote_average,vote_count
0,The Attachment Diaries,El apego,"Argentina, 1970s. A desperate young woman goes...",,"[Drama, Mystery, Thriller, Horror]",0.964,es,[Spanish],[Argentina],102,0,0,2021-10-07,3.000,3
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,Two childhood friends now in their thirties mu...,,"[Romance, Comedy]",35.651,en,"[English, Portuguese, Urdu]","[France, United Kingdom]",109,0,10898395,2023-01-26,6.073,158
2,You Can Live Forever,You Can Live Forever,"When Jaime, a gay teenager, is sent to live in...",https://gooddeedentertainment.com/you-can-live...,"[Drama, Romance]",75.637,en,"[English, French]","[Canada, United States of America]",96,0,15055,2023-03-24,6.583,30
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,"After an unexpected break up, a travel executi...",https://www.netflix.com/title/81424906,"[Romance, Comedy]",23.538,en,"[English, Vietnamese]",[United States of America],96,0,0,2023-04-21,6.313,152
4,Other People’s Children,Les Enfants des autres,"Rachel loves her life, her students, her frien...",https://www.wildbunch.biz/movie/other-peoples-...,"[Drama, Comedy]",12.597,fr,"[English, French]",[France],104,0,84178,2022-09-21,6.876,176


### Merge and Clean the Data for Export

In [83]:
# Merge the New York Times reviews and TMDB DataFrames on title
# Arnab's comments: I am doing a inner join on the "title" column to merge the two dataframes df_reviews and df_movies
# So that I can get the movie information for the those titles which are present in both the dataframes
df_merged = pd.merge(df_movies, df_reviews, on="title", how="inner")
df_merged.head(2)

,title,original_title,overview,homepage,genres,popularity,original_language,spoken_languages,production_countries,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,"Argentina, 1970s. A desperate young woman goes...",,"[Drama, Mystery, Thriller, Horror]",0.964,es,[Spanish],[Argentina],102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,Two childhood friends now in their thirties mu...,,"[Romance, Comedy]",35.651,en,"[English, Portuguese, Urdu]","[France, United Kingdom]",109,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [84]:
# print(df_merged.dtypes)

# Remove list brackets and quotation marks on the columns containing lists

# Arnab's comments: Seeing the output of the sample provided, it seems that quotation mark around the movie name in the
# headline.main column are not removed. So, I am considering only the below 3 columns 
# for removing the brackets and quotation marks 

# Create a list of columns to fix
columns_to_fix = ["genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
chars_to_remove = [ "[", "]", "'" ]


# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    df_merged[column] = df_merged[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        df_merged[column] = df_merged[column].str.replace(char, "")

# Display the dataframe
df_merged.head(2)

,title,original_title,overview,homepage,genres,popularity,original_language,spoken_languages,production_countries,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,"Argentina, 1970s. A desperate young woman goes...",,"Drama, Mystery, Thriller, Horror",0.964,es,Spanish,Argentina,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,Two childhood friends now in their thirties mu...,,"Romance, Comedy",35.651,en,"English, Portuguese, Urdu","France, United Kingdom",109,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [85]:
# Drop "byline.person" column
df_merged = df_merged.drop(columns=["byline.person"])

# Count the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df_merged.shape[0]}")

Number of rows in the DataFrame: 186


In [86]:
# Delete duplicate rows and reset index

# Delete duplicate rows from the DataFrame 
df_merged = df_merged.drop_duplicates()

# Count the number of rows in the DataFrame
print(f"Number of rows in the DataFrame after removing duplicates : {df_merged.shape[0]}")

# Reset the index
df_merged = df_merged.reset_index(drop=True)

# Display the DataFrame
df_merged.head()

Number of rows in the DataFrame after removing duplicates : 184


,title,original_title,overview,homepage,genres,popularity,original_language,spoken_languages,production_countries,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,"Argentina, 1970s. A desperate young woman goes...",,"Drama, Mystery, Thriller, Horror",0.964,es,Spanish,Argentina,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,Two childhood friends now in their thirties mu...,,"Romance, Comedy",35.651,en,"English, Portuguese, Urdu","France, United Kingdom",109,...,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,"When Jaime, a gay teenager, is sent to live in...",https://gooddeedentertainment.com/you-can-live...,"Drama, Romance",75.637,en,"English, French","Canada, United States of America",96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,"After an unexpected break up, a travel executi...",https://www.netflix.com/title/81424906,"Romance, Comedy",23.538,en,"English, Vietnamese",United States of America,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
4,Other People’s Children,Les Enfants des autres,"Rachel loves her life, her students, her frien...",https://www.wildbunch.biz/movie/other-peoples-...,"Drama, Comedy",12.597,fr,"English, French",France,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None


In [87]:
# Export data to a CSV file 'movie_reviews.csv' without the index
# Arnab's comments: If the file already exists, it will be overwritten and not throw any error
df_merged.to_csv("output/movie_reviews.csv", index=False)

